In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-5wnqzzse/kobert-tokenizer_755c4b09606d4991a39363c8384105fd
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-5wnqzzse/kobert-tokenizer_755c4b09606d4991a39363c8384105fd
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=a3cdaed001fe748ec3c0a855145eebaf9e59962a5375e2f4da299e264f5daa8f
  Stored in directory: /tmp/pip-ephem-wheel-cache-615htue_/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [3]:
# requirements
!pip install fastapi uvicorn nest-asyncio pyngrok beautifulsoup4 requests python-dotenv \
  pymysql sqlalchemy tqdm numpy torch pandas gluonnlp transformers \
  sentencepiece mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.6 MB/s eta 0:00:00
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661778 sha256=837edd727d318c536d1f0096447c47abb8c7081b2723d38bbb8ab287470ca8a1
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Attempting u

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/mini_mlops_fastapi')

In [5]:
from fastapi import FastAPI, HTTPException, Depends, status
from fastapi.middleware.cors import CORSMiddleware
from typing import Annotated
from sqlalchemy.orm import Session

from routers.news_scraper import index as scraper
from routers.preprocessor import index as preprocessor
from routers.data_management import index as data_management
from routers.bert_model import index as bert_model
from routers.news_classifier import index as classifier

from models import news_article, preprocessed_article
from database.conn import engine, SessionLocal

In [6]:
app = FastAPI()
news_article.Base.metadata.create_all(bind = engine)
preprocessed_article.Base.metadata.create_all(bind = engine)

In [7]:
# 미들웨어
# CORS 정책
origins = [
    "http://localhost:3000",
    "http://211.62.99.58:3000",
    "http://211.62.99.58:3010",
    ]
app.add_middleware(
    CORSMiddleware,
    allow_origins = origins,
    allow_credentials = True,
    allow_methods = ["*"],
    allow_headers = ["*"],
)

In [8]:
@app.get("/")
def say_hello():
    return {"message": "[Mini MLOps] Hello world from FastAPI."}

# WEB API
app.include_router(data_management.router, prefix="/data_management")

# 수집
app.include_router(scraper.router, prefix="/scraper")

# 전처리(정제)
app.include_router(preprocessor.router, prefix="/preprocessor")

# 학습
app.include_router(bert_model.router, prefix="/model")

# 배포

# 서비스
app.include_router(classifier.router, prefix="/classifier")

In [ ]:
import os
os.environ["NGROK_AUTHTOKEN"] = "2XTROd0DU4gk8fkltlyLyMzqhkm_35diuPKnLNCqoMSfEiYvg"
# os.environ["NGROK_AUTH_TOKEN"] = "2ZLuyiA0f6UNv5uKnapcZpqjo6Y_7wQJxgGfd2m85JHiB6JZ2"
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [322]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://2972-34-125-81-212.ngrok-free.app
INFO:     210.110.33.216:0 - "GET /model/cu HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/fastapi/encoders.py", line 322, in jsonable_encoder
    data = dict(obj)
TypeError: cannot convert dictionary update sequence element #0 to a sequence

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/fastapi/encoders.py", line 327, in jsonable_encoder
    data = vars(obj)
TypeError: vars() argument must have __dict__ attribute

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 408, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 84, in __call__
    return await self.app(scope, receive, send)
  File "/usr/loc